# Information liée à un spectacle 

## Pour un spectacle

In [25]:
import urllib.request
import re
import pandas as pd
# import the BeautifulSoup library so we can parse HTML and XML documents
from bs4 import BeautifulSoup
spectacleId="34525"
#spectacleId="36956"
#spectacleId="20185"
# specify which URL/web page we are going to be scraping
url = "https://www.lesarchivesduspectacle.net/?IDX_Spectacle=" + spectacleId
# open the url using urllib.request and put the HTML into the page variable
page = urllib.request.urlopen(url)
# parse the HTML from our URL into the BeautifulSoup parse tree format
soup = BeautifulSoup(page, "lxml")
#the div where the information is
#the div where the information is
div_interested = soup.find('div', class_='fiche__footer') 
column_label=[]
column_value=[]
for row in div_interested.find_all('table')[0].findAll('tr'):
    th = row.findAll('th')[0].contents[0]
    td = row.findAll('td')[0].contents[0]
    column_label.append(th)
    column_value.append(td)
new_table = pd.DataFrame(columns=column_label, index= [0]) # I know the size
new_table["spectacleId"]=int(spectacleId)
for row in div_interested.find_all('table')[0].findAll('tr'):
    th = row.findAll('th')[0].contents[0]
    td = row.findAll('td')[0].contents[0].strip()
    new_table.at[0,th] = td
#Recuperer creation_date - organism_creation 
#the div where the information is
div_spectacle = soup.find('div', id='div_Spectacle')
p_spectacle = div_spectacle.find_all('p')[0]
if(p_spectacle):
    for time in p_spectacle.findAll('time'):
        if time.has_attr('datetime'):
            new_table["typeDate"]=time.previousSibling
            new_table["date"]=time['datetime']
    for organism in p_spectacle.findAll('a',class_ ='c_Organisme'):
        if organism.has_attr('href'):
            new_table["organismeId"]=organism['href']
            new_table["organismeName"]=organism.contents[0].strip()
new_table["to_analyse"]=p_spectacle.contents[0].strip()
#new_table.dtypes
#table[0] class="f-spectacle__equipe"    table[1]  class="f-spectacle__equipe"  
#table_production = div_spectacle.find_all('table')[2]
#print(table_production)
new_table

,Genre,Durée,Pays,spectacleId,to_analyse
0,Danse,1 heure,France,34525,Création 2011


## Fonction pour récupérer les informations liées à un spectacle


In [66]:
import urllib.request
import re
from bs4 import BeautifulSoup
import pandas as pd


def import_df_type_spectacle(spectacleId):
    # specify which URL/web page we are going to be scraping
    url = "https://www.lesarchivesduspectacle.net/?IDX_Spectacle=" + str(spectacleId)
    try:
        if spectacleId != 0:
            # open the url using urllib.request and put the HTML into the page variable
            page = urllib.request.urlopen(url)
            # parse the HTML from our URL into the BeautifulSoup parse tree format
            soup = BeautifulSoup(page, "lxml")
            div_interested = soup.find('div', class_='fiche__footer') 
            column_label=[]
            column_value=[]
            for row in div_interested.find_all('table')[0].findAll('tr'):
                th = row.findAll('th')[0].contents[0]
                td = row.findAll('td')[0].contents[0]
                column_label.append(th)
                column_value.append(td)
            new_table = pd.DataFrame(columns=column_label, index= [0]) # I know the size
            new_table["spectacleId"]=int(spectacleId)
            for row in div_interested.find_all('table')[0].findAll('tr'):
                th = row.findAll('th')[0].contents[0]
                td = row.findAll('td')[0].contents[0].strip()
                new_table.at[0,th] = td
            #Recuperer creation_date - organism_creation 
            #the div where the information is
            div_spectacle = soup.find('div', id='div_Spectacle')
            p_spectacle = div_spectacle.find_all('p')[0]
            if(p_spectacle):
                for time in p_spectacle.findAll('time'):
                    if time.has_attr('datetime'):
                        new_table["typeDate"]=time.previousSibling
                        new_table["date"]=time['datetime']
                for organism in p_spectacle.findAll('a',class_ ='c_Organisme'):
                    if organism.has_attr('href'):
                        new_table["organismeId"]=organism['href']
                        new_table["organismeName"]=organism.contents[0].strip()
            new_table["to_analyse"]=p_spectacle.contents[0].strip()
            return new_table
    except urllib.error.URLError as e:
        print(e.reason + "for spectacleId "  + str(spectacleId))
    except socket.timeout as e: # <-------- this block here
        print("We timed out")





In [67]:
import urllib.request
import re
from bs4 import BeautifulSoup
import pandas as pd
import time
from datetime import datetime
import pandas as pd
organismId="439"
print("start",datetime.now()) 
csv_file_path = "all_10_years_organism_439.csv"
df = pd.read_csv(csv_file_path)
t_array = df["performanceId"].to_numpy()

frames = [ import_df_type_spectacle(f) for f in t_array ]
result = pd.concat(frames)
result.set_index(['spectacleId'], drop=True, inplace=True)
result.to_csv('all_10_years_organism_' + organismId + '_type_spectacle.csv')


print("finish",datetime.now())



start 2020-07-31 15:20:54.119768
finish 2020-07-31 15:22:32.146880


In [40]:
import urllib.request
import re
from bs4 import BeautifulSoup
import pandas as pd



list_spectacleId=["0","20185","29600","36360"] 
list_df=[]
for spectacleId in list_spectacleId:
    if (spectacleId!=0):
        #call the function to have the type_aide and orgnaism
        try:
            #df_result=import_df_type_spectacle(spectacleId)
            df_one=import_df_type_spectacle(spectacleId)
            list_df=list_df.append(df_one)
        except Exception as err:
            continue
result = pd.concat(list_df)
result.set_index(['spectacleId'], drop=True, inplace=True)
result.to_csv('all_10_years_organism_' + organismId + '_type_spectacle.csv')




TypeError: 'NoneType' object is not iterable

In [43]:
import urllib.request
import re
from bs4 import BeautifulSoup
import pandas as pd


list_spectacleId=["34525","20185","29600","36360"] 
frames = [ import_df_type_spectacle(f) for f in list_spectacleId ]
result = pd.concat(frames)
result.set_index(['spectacleId'], drop=True, inplace=True)
result

,Genre,Durée,Pays,to_analyse,typeDate,date,organismeId,organismeName
spectacleId,,,,,,,,
34525,Danse,1 heure,France,Création 2011,NaN,NaN,NaN,NaN
20185,Danse – à partir de 7 ans,1 heure,France,Création le,Création le,2009-11-03,?IDX_Organisme=837&Saison=2009,Maison de la Danse de Lyon
29600,Danse,1 heure 15,France,Création le,Création le,2010-08-26,?IDX_Organisme=18808&Annee=2010,Festival d'opéra de Vérone
36360,Théâtre,NaN,Pologne,Création le,Création le,2011-10-05,?IDX_Organisme=824&Saison=2011,Le Manège
